In [2]:
from importlib.resources import files

from qch2025.pkg.models.RNN import network as network
from qch2025.pkg.models.RNN import func as func

from qch2025.pkg.dataset import Dataset

import qch2025.pkg.plotting as plt

import numpy as np

n_epochs = 2
n_batches = 200

ds_path = files("qch2025.dataset")

path = ds_path.joinpath("train.csv")
dataset = Dataset(dataset_path=path)

rnn = network.RNN(14, 128, 2)
losses = func.train(rnn, dataset=dataset, epochs=n_epochs,
           batch_size=n_batches, learning_rate=0.01)
print(losses)
func.eval()

plt.plot_line(np.arange(1, n_epochs+1), losses)








Loading Dataset
400
1: avg loss = 0.0022795179486274717
400
2: avg loss = 0.00318008154630661
[0.0022795179486274717, 0.00318008154630661]


AttributeError: module 'qch2025.pkg.plotting' has no attribute 'plot_line'